## Импорт библиотек

In [1]:
from bs4 import BeautifulSoup
import urllib.request
from urllib.parse import unquote
import json
from collections import defaultdict
import requests
import ssl
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.poolmanager import PoolManager
from requests.packages.urllib3.util import ssl_
from bs4 import BeautifulSoup
from urllib.parse import unquote
from datetime import timedelta, date
import re
import time
import sqlite3
import pandas as pd
import time
import matplotlib.pyplot as plt
import numpy as np

## Создаем функции для добычи информации по тегам

Каждая функция будет обращаться к конкретному тегу и доставать оттуда нужную нам информацию. На выходе функции будут выдавать словарь где ключами будут название столбца нашего будущего датафрейма, а значением- значение в столбце.

Функция получения ссылок на объявления

In [2]:
def get_links(url):
    s2=[]
    r = session.request('GET', url)
    soup = BeautifulSoup(r.text, "lxml")
    r1=soup.findAll('a')
    s2.extend(r1)
    links=[]
    
    for i in s2:
        links.append(i.get('href'))
    links=set(filter(lambda x: str(x).startswith('/moskva/kvartiry/1') 
                      or str(x).startswith('/moskva/kvartiry/2') 
                      or str(x).startswith('/moskva/kvartiry/3') 
                      or str(x).startswith('/moskva/kvartiry/4') 
                      or str(x).startswith('/moskva/kvartiry/5')
                      or str(x).startswith('/moskva/kvartiry/6')
                      or str(x).startswith('/moskva/kvartiry/kvartira-studiya'), links))
    
    
    return links

Функции получения информации о метро, количестве комнат и общей площади, общую информацию(площадь кухни, ремонт), информацию о доме, информацию о цене, дате, адресе, координатах.

In [3]:
def get_undeground(soup):
    try:
        metro = soup.find(class_=re.compile('style-item-address-georeferences-item-TZsrp'))
        list_metro=metro.find_all('span')
        list_metro=list(filter(lambda x: x!='',list(map(lambda x: x.text.strip(), list_metro))))
        name=list_metro[0]
        time=list_metro[1]

        return {"metro_name": name, "time_to_metro": time}
    except:
        return {}

In [4]:
def get_title(soup):
    try:
        titles=soup.find('span',{"data-marker":re.compile("item-view/title-info")}).text.strip()
        title = titles.split(',')
        title_list = [title[0],','.join(title[1:-1])]
        title_list=list(map(lambda x: x.strip(), title_list))
        return {"num_rooms": 'Cтудия' if ( title[0].startswith('Кварти')) else title[0][0], "total_area":title_list[1][:4]}
    except:
        return {}

In [5]:
def get_info(soup):
    try:
        dict_titles=dict()
        informations=soup.find_all('li',class_=re.compile('params-paramsList__item-appQw'))
        title_info=list(map(lambda x: x.text.strip().replace('\xa0',''),informations))
        titles=list(map(lambda x: x.find('span').text, informations))
        for i in range(len(title_info)):
            l=title_info[i].replace(titles[i],'')
            dict_titles[titles[i].replace(': ','')]=l

        return dict_titles
    except:
        return {}

In [6]:
def get_info_house(soup):
    try:
        dict_houses=dict()
        house=soup.find_all('li', class_=re.compile('style-item-params-list-item-aXXql'))
        titles_h=list(map(lambda x: x.find('span').text.replace('\xa0',''), house))
        house_info=list(map(lambda x: x.text.strip().replace('\xa0',''),house))
        for i in range(len(house_info)):
            l=house_info[i].replace(titles_h[i],'')
            dict_houses[titles_h[i].replace(':','')]=l
        return dict_houses
    except:
        return {}

In [7]:
def get_price(soup):
    try:
        price=soup.find('span', {'itemprop':'price'})
        return price.text.replace('\xa0','')
    except:
        return ''

In [8]:
def get_date(soup):
    try:
        months = {
        'декабря': '12',
        'ноября': '11',
        'октября': '10',
        'сентября': '9',
        'августа': '8',
        'июля': '07',
        'июня': '06',
        'мая': '05',
        'апреля': '04',
        'марта': '03',
        'февраля': '02',
        'января': '01',
        }
        date_ = soup.find(
                "span", {"data-marker": re.compile('item-view/item-date')}).get_text()
        if 'сегодня' in date_:
            date_ = f"{date.today():%Y-%m-%d}"
        elif 'вчера' in date_:
            date_ = f"{date.today()-timedelta(days=1):%Y-%m-%d}"
        else:
            day = re.search('[0-9]+', date_)[0]
            if len(day) == 1:
                day = '0'+day
            for m in months:
                if m in date_:
                    month = months[m]
                    break
            if month in ['01', '02', '03','04']:
                year = '2023'
            else:
                year = '2022'
            date_ = year+'-'+month+'-'+day

        return date_
    except:
        return ''

In [9]:
def get_address(soup):
    d={}
    try:
        addr=soup.findAll('div', {'itemprop':re.compile("address")})[0].next.next
        
        addr
    except:
        addr=''

    return addr

In [10]:
def get_coords(soup):
    try:
        coords = soup.find(class_=re.compile('style-item-map-wrapper'))
        return    coords['data-map-lat'],    coords['data-map-lon']
    except:
        return ('','')

Функция объединения всех словарей, которые получаются на выходе функций, в один словарь

In [11]:
def get_dict(und, title, inf,house, pr, address, date, link, coords):
    d=dict()
    try:
        d['metro_name']=und['metro_name']
    except KeyError:
        d['metro_name']=''
    try:
        d['time_to_metro']=und['time_to_metro']
    except KeyError:
        d['time_to_metro']=''
    try:
        d['num_rooms']=title['num_rooms']
    except KeyError:
        d['num_rooms']=''
    try:
        d['total_area']=title['total_area']
    except KeyError:
        d['total_area']=''

    try:
        d['year_house']=house['Год постройки']
    except KeyError:
        d['year_house']=''

    try:
        d['kitchen']=inf['Площадь кухни']
    except KeyError:
        d['kitchen']=''
    try:
        d['repair']=inf['Ремонт']
    except KeyError:
        d['repair']=''
    try:
        d['price']=pr
    except KeyError:
        d['price']=''
    try:
        d['floors']=house['Этажей в доме']
    except KeyError:
        d['floors']=''
    try:
        d['num_floor']=inf['Этаж'].split()[0]
    except KeyError:
        d['num_floor']=''
    try:
        d['address']=address
    except KeyError:
        d['address']=''
    try:
        d['date']=date
    except KeyError:
        d['date']=''
    
    d['coord_width'], d['coord_len']=coords
    
    d['link']=f"https://www.avito.ru{link}"

    return d


Класс адаптера и подключение

In [12]:
CIPHERS = """ECDHE-RSA-AES256-GCM-SHA384:ECDHE-ECDSA-AES256-GCM-SHA384:ECDHE-RSA-AES256-SHA384:ECDHE-ECDSA-AES256-SHA384:ECDHE-RSA-AES128-GCM-SHA256:ECDHE-RSA-AES128-SHA256:AES256-SHA"""
class TlsAdapter(HTTPAdapter):

    def __init__(self, ssl_options=0, **kwargs):
        self.ssl_options = ssl_options
        super(TlsAdapter, self).__init__(**kwargs)

    def init_poolmanager(self, *pool_args, **pool_kwargs):
        ctx = ssl_.create_urllib3_context(
            ciphers=CIPHERS, cert_reqs=ssl.CERT_REQUIRED, options=self.ssl_options)
        self.poolmanager = PoolManager(
            *pool_args, ssl_context=ctx, **pool_kwargs)


session = requests.session()
adapter = TlsAdapter(ssl.OP_NO_TLSv1 | ssl.OP_NO_TLSv1_1)
session.mount("https://", adapter)

## Цикл парсинга

data- список словарей, которые в дальнейшем мы преобразуем в пандасовскую табличку. К сожалению ссылки на список объявлений нужно вставлять в ручную. В нашем случае это 6 ссылок- ссылка на студии, однокомнатные и тд., поэтому это не страшно. Приходится в ручную, т.к. Авито меняет расположение индикатора страницы в самой ссылке.

In [13]:
data=[]
for i in range(2,101):
    
    links=get_links(f"https://www.avito.ru/moskva/kvartiry/prodam/vtorichka-ASgBAQICAUSSA8YQAUDmBxSMUg?f=ASgBAQICAUSSA8YQAkDmBxSMUsoIpIpZmqwBmKwBlqwBlKwBiFmGWYRZglmAWQ&p={i}&s=104")
        
    
    for j in links:
        time.sleep(4)
        
        r = session.request('GET', f"https://www.avito.ru{j}")
        soup = BeautifulSoup(r.text, "lxml")
        dict_time=get_dict(get_undeground(soup),get_title(soup),get_info(soup),get_info_house(soup), get_price(soup), get_address(soup), get_date(soup), j, get_coords(soup))
        if list(dict_time.values())==['', '', '', '', '', '', '', '', '', '']:

            
            time.sleep(10)
            
        else:
            data.append(dict_time)
        del soup,r
    time.sleep(4)

## Сохранение данных

Функция преобразования списка словарей в пандасовскую таблицу.

In [14]:
def into_pandas(data):
    df=pd.DataFrame(columns=['metro_name',
                             'time_to_metro',
                             'num_rooms',
                             'total_area',
                             'year_house',
                             'kitchen_area',
                             'repair',
                             'price',
                             'floors',
                             'num_floor', 
                             'address',
                             'date',
                             'coord_width',
                             'coords_len',
                             'link'])
    for i in data:
        df.loc[len(df)]=i.values()
    return df

Сохраняем данные

In [15]:
df_0=into_pandas(data)
df_0.to_csv(r"C:\Users\Stepan\Desktop\ВУЗ\Клиент-серверные\папка датафреймов\1kretry.csv")